In [37]:
import numpy as np
import pandas as pd
import CustFun
import nltk
from collections import Counter


In [38]:
dic={'reviewer_name': str,
        'profile_link': str,
        'country': str,
        'country_label': int,
        'review_count': int,
        'review_date_y': int,
        'review_date_m': int,
        'review_date_d': int,
        'review_date_t': int,
        'rating': int,
        'review_title_text': str,
        'review_title_text_category': int,
        'date_of_experience_y': int,
        'date_of_experience_m': int,
        'date_of_experience_d': int}

In [39]:
df_amzon = pd.read_excel('/Users/alros/Desktop/side01/amzon_review_ok.xlsx',index_col=0,dtype=dic)

### 透過變異數分析，檢查國家與評論分數是否有影響：顯著相關

In [40]:
dic = {country: d['rating'].to_list() for country,d in df_amzon[['country','rating']].groupby('country')}
data = [val for key,val in dic.items()]

In [41]:
#p-value遠小於0.05
print(CustFun.oneway_anvoa(data))

{'F': 3.9381970170186067, 'p-vaule': 1.3217069770093332e-51}


### 整體國家數、評論數、平均分數

In [42]:
#總國家數
df = pd.DataFrame({'總國家數':[len(df_amzon['country'].drop_duplicates())],
                '總評論數':[len(df_amzon)],
                '平均分數':[round(df_amzon['rating'].mean(),2)]})
#母體平均評分
dic=CustFun.get_mean_confidence_interval(df_amzon['rating'])
for key,val in dic.items():
    df[key] = val
df.to_excel('./analyze/total_analyze.xlsx')
df

,總國家數,總評論數,平均分數,p-vaule,lower,upper
0,148,21055,2.19,0.05,2.164004,2.209304


### 列出所有國家的評論數、佔比、平均分數

In [43]:
df = df_amzon['country'].value_counts().to_frame("count")
df['country'] = df.index
df.index  = [i for i in range(0,len(df))]
df["%"] = df['count']/df['count'].sum()*100
df = df.merge(df_amzon[['country','rating']].groupby('country')['rating'].agg('mean'),how='inner',left_on='country',right_index=True)
df['rating'] = np.round(df['rating'],2)
df.to_excel('./analyze/all_country_analyze.xlsx')
df

,count,country,%,rating
0,9286,US,44.103538,2.11
1,7294,GB,34.642603,2.20
2,708,CA,3.362622,1.82
3,629,IN,2.987414,2.27
4,242,IE,1.149371,1.88
...,...,...,...,...
143,1,GF,0.004749,1.00
144,1,KG,0.004749,1.00
145,1,VG,0.004749,3.00
146,1,LA,0.004749,1.00


### 列出佔比大於等於1%的國家的評論數、佔比、平均分數，作為主要分析對象

In [44]:
#評論數>=1%的國家
df = df_amzon['country'].value_counts().to_frame("count")
df['%'] = df['count']/len(df_amzon)*100
lst_country = df.loc[df['%']>=1].index.to_list()
lst_country

['US', 'GB', 'CA', 'IN', 'IE', 'DK', 'NL']

In [45]:
#合併評論數<1%的國家
df = df_amzon.copy()
df.loc[(df['country'].isin(lst_country)==False),['country']] = f'OTHERS(141)'

df2 = df['country'].value_counts().to_frame("count")
df2['%'] = df2['count']/df2['count'].sum()*100

df2 = df2.merge(df[['country','rating']].groupby('country')['rating'].agg('mean').to_frame(),
                how='inner',left_index=True,right_index=True)
df2 = df2.rename(columns={'rating':"mean_rating"})
df2['country'] = df2.index
df2.index = [i for i in range(len(df2))]
df2 = df2[['country','count','%','mean_rating']]
# df2.to_excel('./analyze/country_analyze.xlsx')
df2


,country,count,%,mean_rating
0,US,9286,44.103538,2.105966
1,GB,7294,34.642603,2.198931
2,OTHERS(141),2442,11.598195,2.597052
3,CA,708,3.362622,1.824859
4,IN,629,2.987414,2.268680
5,IE,242,1.149371,1.880165
6,DK,239,1.135122,2.167364
7,NL,215,1.021135,1.911628


### 評論種類比例與評分
0: 遞送與物流體驗 - 主要涉及消費者對配送效率、包裹狀態和配送服務的評價</br>
1: 客戶服務與問題回應 - 聚焦於消費者對客戶服務的評價，特別是對問題解決和退款處理的滿意度</br>
2: 正面購物體驗 - 展現消費者對產品品質、價格和整體購物體驗的高度滿意度</br>
3: 負面服務經歷 - 反映消費者對不良服務體驗的批評，包含客服無回應、退款和產品問題</br>
4: 帳戶與支付問題 - 涉及消費者對帳戶管理、信用卡扣款及支付退款相關的問題與擔憂</br>
5: 產品及服務的不滿意 - 反映消費者對產品質量或服務回應不滿的情緒</br>
6: 優質服務評價 - 描述消費者對優質、快速且可靠服務的正面回饋</br>
7: 商家評價與信任 - 涉及消費者對商家信任度和整體購物體驗的評價，特別是產品與商家聲譽的考量</br>
8: 退貨與退款過程 - 針對退貨及退款流程的評價，關注是否順利完成和處理速度</br>
9: 平台使用體驗 - 反映消費者對購物平台或網站使用體驗的評價，包括操作便捷性和整體滿意度</br>

In [46]:
df_topic = pd.DataFrame({'category':[i for i in range(10)],
                        'topic':['遞送與物流體驗','客戶服務與問題回應','正面購物體驗','負面服務經歷','帳戶與支付問題','產品及服務的不滿意','優質服務評價','商家評價與信任','退貨與退款過程','平台使用體驗'],
                        'summary':['主要涉及消費者對配送效率、包裹狀態和配送服務的評價','聚焦於消費者對客戶服務的評價，特別是對問題解決和退款處理的滿意度','展現消費者對產品品質、價格和整體購物體驗的高度滿意度',
                '反映消費者對不良服務體驗的批評，包含客服無回應、退款和產品問題','涉及消費者對帳戶管理、信用卡扣款及支付退款相關的問題與擔憂','反映消費者對產品質量或服務回應不滿的情緒',
                '描述消費者對優質、快速且可靠服務的正面回饋','涉及消費者對商家信任度和整體購物體驗的評價，特別是產品與商家聲譽的考量','針對退貨及退款流程的評價，關注是否順利完成和處理速度',
                '反映消費者對購物平台或網站使用體驗的評價，包括操作便捷性和整體滿意度']})
# df_topic.to_excel('./analyze/topic_info_analyze.xlsx')
df_topic

,category,topic,summary
0,0,遞送與物流體驗,主要涉及消費者對配送效率、包裹狀態和配送服務的評價
1,1,客戶服務與問題回應,聚焦於消費者對客戶服務的評價，特別是對問題解決和退款處理的滿意度
2,2,正面購物體驗,展現消費者對產品品質、價格和整體購物體驗的高度滿意度
3,3,負面服務經歷,反映消費者對不良服務體驗的批評，包含客服無回應、退款和產品問題
4,4,帳戶與支付問題,涉及消費者對帳戶管理、信用卡扣款及支付退款相關的問題與擔憂
5,5,產品及服務的不滿意,反映消費者對產品質量或服務回應不滿的情緒
6,6,優質服務評價,描述消費者對優質、快速且可靠服務的正面回饋
7,7,商家評價與信任,涉及消費者對商家信任度和整體購物體驗的評價，特別是產品與商家聲譽的考量
8,8,退貨與退款過程,針對退貨及退款流程的評價，關注是否順利完成和處理速度
9,9,平台使用體驗,反映消費者對購物平台或網站使用體驗的評價，包括操作便捷性和整體滿意度


In [47]:
df = df_amzon['review_title_text_category'].value_counts().to_frame('count')
df['category'] = df.index
df = df.sort_values('category')
df.index = [i for i in range(len(df))]
df = df[['category','count']]
df['%'] = [round(c/len(df_amzon)*100,2) for c in df['count'].to_list()]
df['mean_rating'] = [round(df_amzon.loc[df_amzon['review_title_text_category']==i]['rating'].mean(),2) for i in range(0,10)]
df = df.merge(df_topic,how='inner',on='category')
# df.to_excel('./analyze/topic_analyze.xlsx')
df

,category,count,%,mean_rating,topic,summary
0,0,2560,12.16,1.41,遞送與物流體驗,主要涉及消費者對配送效率、包裹狀態和配送服務的評價
1,1,2683,12.74,2.22,客戶服務與問題回應,聚焦於消費者對客戶服務的評價，特別是對問題解決和退款處理的滿意度
2,2,2286,10.86,4.53,正面購物體驗,展現消費者對產品品質、價格和整體購物體驗的高度滿意度
3,3,2300,10.92,1.44,負面服務經歷,反映消費者對不良服務體驗的批評，包含客服無回應、退款和產品問題
4,4,1868,8.87,1.17,帳戶與支付問題,涉及消費者對帳戶管理、信用卡扣款及支付退款相關的問題與擔憂
5,5,1668,7.92,1.81,產品及服務的不滿意,反映消費者對產品質量或服務回應不滿的情緒
6,6,1724,8.19,4.58,優質服務評價,描述消費者對優質、快速且可靠服務的正面回饋
7,7,2273,10.80,1.51,商家評價與信任,涉及消費者對商家信任度和整體購物體驗的評價，特別是產品與商家聲譽的考量
8,8,3051,14.49,1.31,退貨與退款過程,針對退貨及退款流程的評價，關注是否順利完成和處理速度
9,9,642,3.05,3.49,平台使用體驗,反映消費者對購物平台或網站使用體驗的評價，包括操作便捷性和整體滿意度


### 列舉主要國家：評論分佈、各別評分、低分的問題集中在哪

In [48]:
nltk.download('stopwords')
lst_stopwords = nltk.corpus.stopwords.words('english')
lst_stopwords.append('amazon')

[nltk_data] Downloading package stopwords to /Users/alros/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
def country_info(data:pd.DataFrame,country,df_topic,stop_words):
    df = data[data['country']==country]
    print(country)
    print('平均評分',df['rating'].mean())

    df2 = df['review_title_text_category'].value_counts().to_frame('count')
    df2['category'] = df2.index
    df2 = df2.sort_values('category')

    df2 = df2.merge(df[['review_title_text_category','rating']].groupby('review_title_text_category')['rating'].agg('mean'),how='left',left_on='category',right_index=True)
    df2 = df2.rename(columns={'rating':'mean_rating'})

    df2 = df2.merge(data[['review_title_text_category','rating']].groupby('review_title_text_category')['rating'].agg('mean'),how='left',left_on='category',right_index=True)
    df2 = df2.rename(columns={'rating':'total_mean_rating'})

    df2['%'] = [c/ df2['count'].sum()*100 for c in df2['count']]
    df2 = df2[['category','%','count','mean_rating','total_mean_rating']]
    df2 = df2.merge(df_topic[['category','topic']],how='inner',on='category')

    lst = []
    for i in range(10):
        s = CustFun.convert(df.loc[df['review_title_text_category']==i,['review_title_text']]['review_title_text'].to_list()
                            ,stop_words)
        word_counts = Counter(s)
        lst.append(",".join([ word for word, count in word_counts.most_common(50)]))
    df2['key_word_top_50'] = lst
    return df2


In [50]:
df_amzon.loc[(df_amzon['country'].isin(lst_country)==False),['country']] = 'OTHERS(141)'

In [51]:
dic = {}
for c in lst_country:
    df = country_info(df_amzon,c,df_topic,lst_stopwords)
    dic[c] = df
    # df.to_excel(f'./analyze/{c}_analyze.xlsx')
c='OTHERS(141)'
df = country_info(df_amzon,c,df_topic,lst_stopwords)
dic[c] = df

US
平均評分 2.1059659702778375
GB
平均評分 2.1989306279133536
CA
平均評分 1.8248587570621468
IN
平均評分 2.268680445151033
IE
平均評分 1.8801652892561984
DK
平均評分 2.1673640167364017
NL
平均評分 1.9116279069767441
OTHERS(141)
平均評分 2.597051597051597


### 取得US和GB評論中前500出現頻率最高的關鍵字

In [52]:
def getMaxReviewWordsFeq(counrty_data,data:pd.DataFrame,country:str,stop_words)->pd.DataFrame:
    # category = counrty_data.loc[counrty_data['%'].max() == counrty_data['%']]['category'].to_list()[0]
    words = CustFun.convert(data.loc[(data['country']==country),['review_title_text']]['review_title_text'].to_list(),stop_words)

    dic = {}
    for w in words: 
        if w not in dic.keys():
            dic[w] = [1] 
        else: 
            dic[w][0]+=1

    df = pd.DataFrame(data=dic)

    df = df.T
    df = df.rename(columns={df.columns[0]:'count'})
    df['words'] = df.index
    df = df.sort_values(['count'],ascending=False)
    df.index = [i for i in range(0,len(df))]
    df = df.iloc[:500]

    return df

In [53]:
df = getMaxReviewWordsFeq(dic['US'],df_amzon,'US',lst_stopwords)
df.to_excel('./analyze/us_words_feq.xlsx')
df

,count,words
0,8743,wa
1,7403,customer
2,6731,service
3,5403,item
4,4591,order
...,...,...
495,165,hate
496,165,delay
497,165,warehouse
498,164,c


In [54]:
df = getMaxReviewWordsFeq(dic['GB'],df_amzon,'GB',lst_stopwords)
df.to_excel('./analyze/gb_words_feq.xlsx')
df

,count,words
0,5435,wa
1,5027,service
2,4563,delivery
3,4408,customer
4,3620,item
...,...,...
495,115,cancelling
496,115,idea
497,114,friday
498,114,department
